In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# import requests
# from config import weather_api_key


In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=1500)
lngs = np.random.uniform(-180, 180, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the lats and longs to a list
coordinates = list(lat_lngs)

In [4]:
#Create a list for holding the cities
cities=[]
#Identify the nearest city for each lat and lon combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#Print the city count tto confirm sufficient count.
#len(cities)
#print(cities)


In [5]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
#print(weather_api_key)

ModuleNotFoundError: No module named 'config'

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key
print(url)


In [ ]:
# # Create an endpoint URL for a city.
city_url = url + "&q=" + "boston"
print(city_url)

In [ ]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime


In [ ]:
# CREate an empty list to hold the weather data
city_data = []
#print the beginning to the logging.
print("Beginning Data Retrieval    ")
print("----------------------------")
#create counters
record_count = 1
set_count = 1

#Loop through all the cities in our list.
for i, city in enumerate(cities):
    #Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count +=1
        record_count =1
        time.sleep(60)
        
    #Create andpoint url with each city
    #print(city)
    city_url = url + "&q=" + city.replace(" ","+")
    #With Reza, worked to find problems with API key using the print function below.)
    #print(city_url)

    #Log the url, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #Add 1 to the record count
    record_count +=1
    
#Run and API request for each of the cities
    try:
        #Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #parse the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_winds = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        #convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #append the city info into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_winds,
                          "Country": city_country,
                          "Date": city_date
        })
        
#If an error is experienced, skip the city
    except Exception as e:
        print(e)
        print("City not found. Skipping...")
        pass
    
#Indicate that Data Loading is complete
print("----------------------------")
print("Data Retrieval Complete     ")
print("----------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
new_column_order = ["City", "Country", "Date", "Lat","Lng","Max Temp", "Humidity", "Cloudiness", "Wind Speed"]

In [ ]:
df = city_data_df[new_column_order]

In [ ]:
df


In [ ]:
# Create the output file (CSV).
output_data_file = "World_Weather_Analysis/weather_data/cities.csv"
#export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")